In [2]:
# Citire date

# Dupa ce antrenez modelul pe datele de train si testez pe cele de validare, pentru submisia finala
# folosesc si datele de validare ca date de antrenare. Aici unesc fisierele
import pandas as pd

filenames = ['ProiectML\\train_samples.txt','ProiectML\\validation_samples.txt']
with open('ProiectML\\my_train_samples.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)
                
filenames = ['ProiectML\\train_labels.txt','ProiectML\\validation_labels.txt']
with open('ProiectML\\my_train_labels.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)
                
df = pd.read_fwf("ProiectML\\my_train_samples.txt", sep='\t', header=None)
df_labels = pd.read_csv("ProiectML\\my_train_labels.txt", sep='\t', header=None)

df_merged = pd.merge(df, df_labels, left_on=0, right_on=0) # Inner join intre samples si labels (dupa col 0)

In [4]:
# https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
# Tochenizare - construiesc un dictionar de features si transorm doc in vectori de features
from sklearn.feature_extraction.text import CountVectorizer 
cvect = CountVectorizer(ngram_range=(1, 2))   
train_data1 = cvect.fit_transform(df_merged['1_x'].values)
  
# Term frequencies - pana acum aveam aparitia cuv, acum iau frecventa
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(norm= 'l2', use_idf= False)
train_data = tfidf_transformer.fit_transform(train_data1)

# Definesc modelul
from sklearn.naive_bayes import MultinomialNB  
cls = MultinomialNB(alpha = 0.1)
cls.fit(train_data, df_merged['1_y'].values) # Antrenez modelul

# Aplic pentru test
df_test = pd.read_fwf("ProiectML\\test_samples.txt", sep='\t', header=None)

test_data1 = cvect.transform(df_test[1].values)
test_data = tfidf_transformer.transform(test_data1)
prediction = cls.predict(test_data)

final_df = pd.DataFrame({'id': df_test[0].values,
                         'label': prediction
                        })

final_df.to_csv('SUBM_PtMAINE_OFICIAL.csv', index=False)